Crear Carpeta Mes ACTUAL

COPIAR FORECAST AUNQUE ESTE ABIERTO (FORECAST ACTUAL, editar ruta de origen para el pasado si es necesario)

In [1]:
import pandas as pd
import os
from datetime import datetime
import shutil
import openpyxl

ruta_origen = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Forecast Inbound\2024-12 Diciembre\12.2024 S&OP Demanda Sin restricciones Inbound Ciclo Enero 25_Inbound.xlsx"

def find_base_sheet(ruta_excel):
    xls = pd.ExcelFile(ruta_excel)
    for sheet in xls.sheet_names:
        if sheet.lower() == 'base':
            return sheet
    return None

def clean_dataframe(df):
    df.columns = df.columns.astype(str)
    for column in df.columns:
        if df[column].dtype == 'object':
            try:
                df[column] = df[column].astype(str)
            except:
                pass
    return df

def excel_to_parquet(ruta_excel, ruta_parquet):
    base_sheet = find_base_sheet(ruta_excel)
    if base_sheet:
        df = pd.read_excel(ruta_excel, sheet_name=base_sheet, skiprows=2, engine='openpyxl')
        df = clean_dataframe(df)
        df.to_parquet(ruta_parquet)
    else:
        print(f"No se encontró la hoja 'BASE' en el archivo {ruta_excel}")

usuario = os.getlogin()
anio_actual = datetime.now().year
mes_actual = datetime.now().month

ruta_guardado_excel = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\PWBI Forecast\Evolutivo"
ruta_guardado_parquet = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\PWBI Forecast"

# Crear las carpetas si no existen
if not os.path.exists(ruta_guardado_excel):
    os.makedirs(ruta_guardado_excel)

if not os.path.exists(ruta_guardado_parquet):
    os.makedirs(ruta_guardado_parquet)

nombre_archivo = os.path.basename(ruta_origen)
ruta_copiada = os.path.join(ruta_guardado_excel, nombre_archivo)

# Copiar el archivo utilizando shutil, aunque esté abierto
shutil.copy2(ruta_origen, ruta_copiada)

nombre_parquet_actual = os.path.splitext(nombre_archivo)[0] + "_Forecast.parquet"
ruta_parquet_actual = os.path.join(ruta_guardado_parquet, nombre_parquet_actual)
excel_to_parquet(ruta_copiada, ruta_parquet_actual)


# LeerParquetFC Actual y Anterior

In [ ]:
import os
import pandas as pd
from getpass import getuser

usuario = getuser()
ruta_directorio = fr"C:\Users\{usuario}\Inchcape\Planificación y abastecimiento AFM - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\PWBI Forecast"
archivos = os.listdir(ruta_directorio)
archivos_forecast = [archivo for archivo in archivos if archivo.endswith('.parquet') and 'Forecast' in archivo]
archivos_con_fecha = [archivo for archivo in archivos_forecast if archivo[:7].replace('.', '').isdigit()]
archivos_con_fecha.sort(key=lambda x: pd.to_datetime(x[:7], format='%m.%Y'), reverse=True)

archivo_mas_reciente = archivos_con_fecha[0]
ruta_archivo_mas_reciente = os.path.join(ruta_directorio, archivo_mas_reciente)
fcactual = pd.read_parquet(ruta_archivo_mas_reciente)

fecha_actual = pd.to_datetime(archivo_mas_reciente[:7], format='%m.%Y')
fecha_anterior = (fecha_actual - pd.DateOffset(months=1)).strftime('%m.%Y')

archivo_mes_anterior = [archivo for archivo in archivos_con_fecha if archivo.startswith(fecha_anterior)]
if archivo_mes_anterior:
    ruta_archivo_mes_anterior = os.path.join(ruta_directorio, archivo_mes_anterior[0])
    fcanterior = pd.read_parquet(ruta_archivo_mes_anterior)
else:
    fcanterior = None
fcactual.drop(columns=['Material R3'], inplace=True)
fcanterior.drop(columns=['Material R3'], inplace=True)

print("Ruta del Forecast Actual:", ruta_archivo_mas_reciente)
print("Ruta del Forecast Anterior:", ruta_archivo_mes_anterior if archivo_mes_anterior else "No se encontró el archivo del mes anterior.")


In [ ]:
fcactual.head(1)

In [ ]:
fcactual['Sell Out'] = None
fcactual['FC BASE'] = None
fcactual['Ajustado'] = None
fcactual['Ciclo Anterior'] = None
fcactual['stock'] = None
fcactual['No Planificable'] = None
fcactual['FC Final'] = None
fcactual['fc valorizado'] = None
fcactual['fc accion'] = None

columnas = fcactual.columns.tolist()
columnas.insert(32, columnas.pop(columnas.index('Sell Out')))
columnas.insert(57, columnas.pop(columnas.index('FC BASE')))
columnas.insert(77, columnas.pop(columnas.index('Ajustado')))
columnas.insert(90, columnas.pop(columnas.index('Ciclo Anterior')))
columnas.insert(103, columnas.pop(columnas.index('stock')))
columnas.insert(126, columnas.pop(columnas.index('No Planificable')))
columnas.insert(151, columnas.pop(columnas.index('fc accion')))
columnas.insert(176, columnas.pop(columnas.index('FC Final')))
columnas.insert(189, columnas.pop(columnas.index('fc valorizado')))

fcactual = fcactual[columnas]

In [ ]:
from datetime import datetime, timedelta

fecha_actual = datetime.now()
hace_2_meses = (fecha_actual - timedelta(days=60)).strftime('%Y-%m')
mes_siguiente = (fecha_actual + timedelta(days=30)).strftime('%Y-%m')
mes_pasado = (fecha_actual - timedelta(days=30)).strftime('%Y-%m')

validaciones = {
    'Sell Out': hace_2_meses,
    'FC BASE': mes_siguiente,
    'Ajustado': mes_siguiente,
    'No Planificable': mes_siguiente,
    'FC Final': mes_siguiente,
    'fc valorizado': mes_siguiente,
    'fc accion': mes_siguiente,
    'Ciclo Anterior': mes_pasado
}

for columna, fecha in validaciones.items():
    index = fcactual.columns.get_loc(columna)
    if columna == 'Sell Out':
        columna_index = index + 1
    else:
        columna_index = index + 1

    if columna_index < len(fcactual.columns) and fcactual.columns[columna_index] != fecha:
        print(f'La primera columna después de "{columna}" ({fcactual.columns[columna_index]})')


In [ ]:
from datetime import datetime, timedelta

fecha_actual = datetime.now()
hace_2_meses = (fecha_actual - timedelta(days=60)).strftime('%Y-%m')
mes_siguiente = (fecha_actual + timedelta(days=30)).strftime('%Y-%m')
mes_pasado = (fecha_actual - timedelta(days=30)).strftime('%Y-%m')

validaciones = {
    'Sell Out': hace_2_meses,
    'FC BASE': mes_siguiente,
    'Ajustado': mes_siguiente,
    'No Planificable': mes_siguiente,
    'FC Final': mes_siguiente,
    'fc valorizado': mes_siguiente,
    'fc accion': mes_siguiente,
    'Ciclo Anterior': mes_pasado
}

for columna, fecha in validaciones.items():
    index = fcactual.columns.get_loc(columna)
    if columna == 'Sell Out':
        columna_index = index + 24
    else:
        columna_index = index + 12

    if columna_index < len(fcactual.columns) and fcactual.columns[columna_index] != fecha:
        print(f'La ultima columna de "{columna}" ({fcactual.columns[columna_index]})')


In [ ]:
# Seleccionar las primeras 15 column
pri = fcactual.columns[125:152].tolist()
pri


# Creando DFs para el FC ACTUAL

In [ ]:

if 'ID' not in fcactual.columns:
    raise ValueError("La columna 'ID' no se encuentra en el DataFrame")
columnas_sellout = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[fcactual.columns.get_loc('Canal')+1:fcactual.columns.get_loc('Canal')+19].tolist()
sellin = fcactual[columnas_sellout]

indice_fcbase = fcactual.columns.get_loc('FC BASE') + 1
columnas_fcbase = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_fcbase:indice_fcbase+12].tolist()
fcbase = fcactual[columnas_fcbase]

indice_fcajustado = fcactual.columns.get_loc('Ajustado') + 1
columnas_fcajustado = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_fcajustado:indice_fcajustado+12].tolist()
fcajustado = fcactual[columnas_fcajustado]

indice_ciclo_anterior = fcactual.columns.get_loc('Ciclo Anterior') + 1
columnas_cicloanterior = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_ciclo_anterior:indice_ciclo_anterior+12].tolist()
cicloanterior = fcactual[columnas_cicloanterior]

indice_stock = fcactual.columns.get_loc('stock') + 1
columnas_stock = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_stock:indice_stock+9].tolist()
stock = fcactual[columnas_stock]

indice_no_planificable = fcactual.columns.get_loc('No Planificable') + 1
columnas_noplanificable = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_no_planificable:indice_no_planificable+12].tolist()
noplanificable = fcactual[columnas_noplanificable]

indice_fcfinal = fcactual.columns.get_loc('FC Final') + 1
columnas_fcfinal = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_fcfinal:indice_fcfinal+12].tolist()
fcfinal = fcactual[columnas_fcfinal]

indice_fcvalorizado = fcactual.columns.get_loc('fc valorizado') + 1
columnas_fcvalorizado = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_fcvalorizado:indice_fcvalorizado+12].tolist()
fcvalorizado = fcactual[columnas_fcvalorizado]

indice_fcaccion = fcactual.columns.get_loc('fc accion') + 1
columnas_fcaccion = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_fcaccion:indice_fcaccion+12].tolist()
fcaccion = fcactual[columnas_fcaccion]

indice_selloutx = fcactual.columns.get_loc('Sell Out') + 1
columnas_selloutx = [fcactual.columns[fcactual.columns.get_loc('ID')]] + fcactual.columns[indice_selloutx:indice_selloutx+24].tolist()
sellout = fcactual[columnas_selloutx]

fc_category = fcactual.iloc[:, :14]

dataframes = [
    (sellin, 'sellin'),
    (fcbase, 'fcbase'),
    (fcajustado, 'fcajustado'),
    (cicloanterior, 'cicloanterior'),
    (stock, 'stock'),
    (noplanificable, 'noplanificable'),
    (fcfinal, 'fcfinal'),
    (fcvalorizado, 'fcvalorizado'),
    (fcaccion, 'fcaccion'),
    (sellout, 'sellout'),
    (fc_category, 'fc_category')
]

for df, nombre in dataframes:
    print(f"{nombre} tiene {df.shape[1]} columnas")

In [ ]:
noplanificable.columns

# MELT FORECAST ACTUAL

In [ ]:
sellin_melt = sellin.melt(id_vars=[sellin.columns[0]], var_name='Fecha_sellin', value_name='Valor')
fcbase_melt = fcbase.melt(id_vars=[fcbase.columns[0]], var_name='Fecha_fcbase', value_name='Valor')
fcajustado_melt = fcajustado.melt(id_vars=[fcajustado.columns[0]], var_name='Fecha_fcajustado', value_name='Valor')
cicloanterior_melt = cicloanterior.melt(id_vars=[cicloanterior.columns[0]], var_name='Fecha_cicloanterior', value_name='Valor')
stock_melt = stock.melt(id_vars=[stock.columns[0]], var_name='Fecha_stock', value_name='Valor')
noplanificable_melt = noplanificable.melt(id_vars=[noplanificable.columns[0]], var_name='Fecha_noplanificable', value_name='Valor')
fcfinal_melt = fcfinal.melt(id_vars=[fcfinal.columns[0]], var_name='Fecha_fcfinal', value_name='Valor')
fcvalorizado_melt = fcvalorizado.melt(id_vars=[fcvalorizado.columns[0]], var_name='Fecha_fcvalorizado', value_name='Valor')
fcaccion_melt = fcaccion.melt(id_vars=[fcaccion.columns[0]], var_name='Fecha_fcaccion', value_name='Valor')
sellout_melt = sellout.melt(id_vars=[sellout.columns[0]], var_name='Fecha_sellout', value_name='Valor')
sellin_melt['Fecha_sellin'] = sellin_melt['Fecha_sellin'].str[:10]
fcbase_melt['Fecha_fcbase'] = fcbase_melt['Fecha_fcbase'].str[:10]
fcajustado_melt['Fecha_fcajustado'] = fcajustado_melt['Fecha_fcajustado'].str[:10]
cicloanterior_melt['Fecha_cicloanterior'] = cicloanterior_melt['Fecha_cicloanterior'].str[:10]
stock_melt['Fecha_stock'] = stock_melt['Fecha_stock'].str[:10]
noplanificable_melt['Fecha_noplanificable'] = noplanificable_melt['Fecha_noplanificable'].str[:10]
fcfinal_melt['Fecha_fcfinal'] = fcfinal_melt['Fecha_fcfinal'].str[:10]
fcvalorizado_melt['Fecha_fcvalorizado'] = fcvalorizado_melt['Fecha_fcvalorizado'].str[:10]
fcaccion_melt['Fecha_fcaccion'] = fcaccion_melt['Fecha_fcaccion'].str[:10]
sellout_melt['Fecha_sellout'] = sellout_melt['Fecha_sellout'].str[:10]
sellin_melt['Fecha_sellin'] = pd.to_datetime(sellin_melt['Fecha_sellin'])
fcbase_melt['Fecha_fcbase'] = pd.to_datetime(fcbase_melt['Fecha_fcbase'])
fcajustado_melt['Fecha_fcajustado'] = pd.to_datetime(fcajustado_melt['Fecha_fcajustado'])
cicloanterior_melt['Fecha_cicloanterior'] = pd.to_datetime(cicloanterior_melt['Fecha_cicloanterior'])
stock_melt['Fecha_stock'] = pd.to_datetime(stock_melt['Fecha_stock'])
noplanificable_melt['Fecha_noplanificable'] = pd.to_datetime(noplanificable_melt['Fecha_noplanificable'])
fcfinal_melt['Fecha_fcfinal'] = pd.to_datetime(fcfinal_melt['Fecha_fcfinal'])
fcvalorizado_melt['Fecha_fcvalorizado'] = pd.to_datetime(fcvalorizado_melt['Fecha_fcvalorizado'])
fcaccion_melt['Fecha_fcaccion'] = pd.to_datetime(fcaccion_melt['Fecha_fcaccion'])
sellout_melt['Fecha_sellout'] = pd.to_datetime(sellout_melt['Fecha_sellout'])

In [ ]:

# Renombrar las columnas que empiezan con 'Fecha' a 'Fecha'
fecha_sellout_melt = sellout_melt.rename(columns={col: 'Fecha' for col in sellout_melt.columns if col.startswith('Fecha')})[['ID', 'Fecha']]
fecha_fc_anterior = cicloanterior_melt.rename(columns={col: 'Fecha' for col in cicloanterior_melt.columns if col.startswith('Fecha')})[['ID', 'Fecha']]
fecha_fc_final = fcfinal_melt.rename(columns={col: 'Fecha' for col in fcfinal_melt.columns if col.startswith('Fecha')})[['ID', 'Fecha']]

# Concatenar las columnas 'Fecha' y 'ID'
fechas_concatenadas = pd.concat([fecha_sellout_melt, fecha_fc_anterior, fecha_fc_final], axis=0)
fechas_concatenadas.drop_duplicates(subset=['ID', 'Fecha'], inplace=True)
fechas_concatenadas.reset_index(drop=True, inplace=True)
fechas_concatenadas.shape


In [ ]:
fechas_concatenadas

In [ ]:
# print("sellin_melt:")
# print(sellin_melt.head(2))
# print("fcbase_melt:")
# print(fcbase_melt.head(2))
# print("fcajustado_melt:")
# print(fcajustado_melt.head(2))
# print("cicloanterior_melt:")
# print(cicloanterior_melt.head(2))
# print("stock_melt:")
# print(stock_melt.head(2))
# print("noplanificable_melt:")
# print(noplanificable_melt.head(2))
# print("fcfinal_melt:")
# print(fcfinal_melt.head(2))
# print("fcvalorizado_melt:")
# print(fcvalorizado_melt.head(2))
# print("fcaccion_melt:")
# print(fcaccion_melt.head(2))
# print("sellout_melt:")
# print(sellout_melt.head(2))

In [ ]:

# Cambiar el nombre de las columnas 'ID' y 'Valor' en cada DataFrame
sellin_melt = sellin_melt.rename(columns={'ID': 'ID_sellin', 'Valor': 'Valor_sellin'})
fcbase_melt = fcbase_melt.rename(columns={'ID': 'ID_fcbase', 'Valor': 'Valor_fcbase'})
fcajustado_melt = fcajustado_melt.rename(columns={'ID': 'ID_fcajustado', 'Valor': 'Valor_fcajustado'})
cicloanterior_melt = cicloanterior_melt.rename(columns={'ID': 'ID_cicloanterior', 'Valor': 'Valor_cicloanterior'})
stock_melt = stock_melt.rename(columns={'ID': 'ID_stock', 'Valor': 'Valor_stock'})
noplanificable_melt = noplanificable_melt.rename(columns={'ID': 'ID_noplanificable', 'Valor': 'Valor_noplanificable'})
fcfinal_melt = fcfinal_melt.rename(columns={'ID': 'ID_fcfinal', 'Valor': 'Valor_fcfinal'})
fcvalorizado_melt = fcvalorizado_melt.rename(columns={'ID': 'ID_fcvalorizado', 'Valor': 'Valor_fcvalorizado'})
fcaccion_melt = fcaccion_melt.rename(columns={'ID': 'ID_fcaccion', 'Valor': 'Valor_fcaccion'})
sellout_melt = sellout_melt.rename(columns={'ID': 'ID_sellout', 'Valor': 'Valor_sellout'})

# Concatenar los DataFrames horizontalmente (a la derecha)
df_concatenado = pd.concat([sellin_melt, fcbase_melt, fcajustado_melt, cicloanterior_melt, stock_melt,
                            noplanificable_melt, fcfinal_melt, fcvalorizado_melt, fcaccion_melt, sellout_melt], axis=1)

# Guardar el DataFrame concatenado en un archivo parquet
df_concatenado.to_parquet(os.path.join(folder_path, 'df_concatenado.parquet'))

print("Archivo parquet concatenado guardado en la carpeta:", folder_path)


In [ ]:
# Guardar el DataFrame concatenado en un archivo parquet
df_concatenado.to_parquet(os.path.join(folder_path, 'data_concatenada.parquet'))

print("Archivo parquet concatenado guardado en la carpeta:", folder_path)


In [ ]:
sellin_melt.dtypes

# CREANDO COLUMNAS EN FC ANTERIOR

In [ ]:
fcanterior['Sell Out'] = None
fcanterior['FC BASE'] = None
fcanterior['Ajustado'] = None
fcanterior['Ciclo Anterior'] = None
fcanterior['stock'] = None
fcanterior['No Planificable'] = None
fcanterior['FC Final'] = None
fcanterior['fc valorizado'] = None
fcanterior['fc accion'] = None

columnas = fcanterior.columns.tolist()
columnas.insert(32, columnas.pop(columnas.index('Sell Out')))
columnas.insert(57, columnas.pop(columnas.index('FC BASE')))
columnas.insert(77, columnas.pop(columnas.index('Ajustado')))
columnas.insert(90, columnas.pop(columnas.index('Ciclo Anterior')))
columnas.insert(103, columnas.pop(columnas.index('stock')))
columnas.insert(126, columnas.pop(columnas.index('No Planificable')))
columnas.insert(175, columnas.pop(columnas.index('FC Final')))
columnas.insert(188, columnas.pop(columnas.index('fc valorizado')))
columnas.insert(151, columnas.pop(columnas.index('fc accion')))
fcanterior = fcanterior[columnas]


# ALERTAS que valida las columnas#

In [ ]:
from datetime import datetime, timedelta

fecha_actual = datetime.now()
hace_2_meses = (fecha_actual - timedelta(days=60)).strftime('%Y-%m')
mes_siguiente = (fecha_actual + timedelta(days=30)).strftime('%Y-%m')
mes_pasado = (fecha_actual - timedelta(days=30)).strftime('%Y-%m')

validaciones = {
    'Sell Out': hace_2_meses,
    'FC BASE': mes_siguiente,
    'Ajustado': mes_siguiente,
    'No Planificable': mes_siguiente,
    'FC Final': mes_siguiente,
    'fc valorizado': mes_siguiente,
    'fc accion': mes_siguiente,
    'Ciclo Anterior': mes_pasado
}

for columna, fecha in validaciones.items():
    index = fcanterior.columns.get_loc(columna)
    if columna == 'Sell Out':
        columna_index = index + 24
    else:
        columna_index = index + 1

    if columna_index < len(fcanterior.columns) and fcanterior.columns[columna_index] != fecha:
        print(f'La primera columna después de "{columna}" ({fcanterior.columns[columna_index]}) no es igual a {fecha}')


# Creando DFs para el FC Anterior

In [ ]:
columnas_sellout_anterior = [fcanterior.columns[0]] + fcanterior.columns[fcanterior.columns.get_loc('Canal')+1:fcanterior.columns.get_loc('Canal')+19].tolist()
sellin_anterior = fcanterior[columnas_sellout_anterior]

indice_fcbase_anterior = fcanterior.columns.get_loc('FC BASE') + 1
columnas_fcbase_anterior = [fcanterior.columns[0]] + fcanterior.columns[indice_fcbase_anterior:indice_fcbase_anterior+12].tolist()
fcbase_anterior = fcanterior[columnas_fcbase_anterior]

indice_fcajustado_anterior = fcanterior.columns.get_loc('Ajustado') + 1
columnas_fcajustado_anterior = [fcanterior.columns[0]] + fcanterior.columns[indice_fcajustado_anterior:indice_fcajustado_anterior+12].tolist()
fcajustado_anterior = fcanterior[columnas_fcajustado_anterior]

indice_ciclo_anterior_anterior = fcanterior.columns.get_loc('Ciclo Anterior') + 1
columnas_cicloanterior_anterior = [fcanterior.columns[0]] + fcanterior.columns[indice_ciclo_anterior_anterior:indice_ciclo_anterior_anterior+12].tolist()
cicloanterior_anterior = fcanterior[columnas_cicloanterior_anterior]

indice_stock_anterior = fcanterior.columns.get_loc('stock') + 1
columnas_stock_anterior = [fcanterior.columns[0]] + fcanterior.columns[indice_stock_anterior:indice_stock_anterior+9].tolist()
stock_anterior = fcanterior[columnas_stock_anterior]

indice_no_planificable_anterior = fcanterior.columns.get_loc('No Planificable') + 1
columnas_noplanificable_anterior = [fcanterior.columns[0]] + fcanterior.columns[indice_no_planificable_anterior:indice_no_planificable_anterior+12].tolist()
noplanificable_anterior = fcanterior[columnas_noplanificable_anterior]

indice_fcfinal_anterior = fcanterior.columns.get_loc('FC Final') + 1
columnas_fcfinal_anterior = [fcanterior.columns[0]] + fcanterior.columns[indice_fcfinal_anterior:indice_fcfinal_anterior+12].tolist()
fcfinal_anterior = fcanterior[columnas_fcfinal_anterior]

indice_fcvalorizado_anterior = fcanterior.columns.get_loc('fc valorizado') + 1
columnas_fcvalorizado_anterior = fcanterior.columns[indice_fcvalorizado_anterior:indice_fcvalorizado_anterior+12].tolist()
fcvalorizado_anterior = fcanterior[columnas_fcvalorizado_anterior]

indice_fcaccion_anterior = fcanterior.columns.get_loc('fc accion') + 1
columnas_fcaccion_anterior = fcanterior.columns[indice_fcaccion_anterior:indice_fcaccion_anterior+12].tolist()
fcaccion_anterior = fcanterior[columnas_fcaccion_anterior]
indice_selloutx = fcanterior.columns.get_loc('Sell Out') + 1
columnas_selloutx = [fcanterior.columns[0]] + fcanterior.columns[indice_selloutx:indice_selloutx+24].tolist()
selloutA = fcanterior[columnas_selloutx]



In [ ]:
fcaccion_anterior.columns

# MELT ANTERIOR

In [ ]:
sellin_anterior_melt = sellin_anterior.melt(id_vars=[sellin_anterior.columns[0]], var_name='Fecha', value_name='Valor')

fcbase_anterior_melt = fcbase_anterior.melt(id_vars=[fcbase_anterior.columns[0]], var_name='Fecha', value_name='Valor')

fcadj_anterior_melt = fcajustado_anterior.melt(id_vars=[fcajustado_anterior.columns[0]], var_name='Fecha', value_name='Valor')

cicloant_anterior_melt = cicloanterior_anterior.melt(id_vars=[cicloanterior_anterior.columns[0]], var_name='Fecha', value_name='Valor')

stock_anterior_melt = stock_anterior.melt(id_vars=[stock_anterior.columns[0]], var_name='Fecha', value_name='Valor')

noplan_anterior_melt = noplanificable_anterior.melt(id_vars=[noplanificable_anterior.columns[0]], var_name='Fecha', value_name='Valor')

fcfin_anterior_melt = fcfinal_anterior.melt(id_vars=[fcfinal_anterior.columns[0]], var_name='Fecha', value_name='Valor')

fcval_anterior_melt = fcvalorizado_anterior.melt(id_vars=[fcvalorizado_anterior.columns[0]], var_name='Fecha', value_name='Valor')

fcacc_anterior_melt = fcaccion_anterior.melt(id_vars=[fcaccion_anterior.columns[0]], var_name='Fecha', value_name='Valor')

sellout_anterior_melt = selloutA.melt(id_vars=[selloutA.columns[0]], var_name='Fecha', value_name='Valor')

In [ ]:
import pandas as pd
import re

# Función para limpiar fechas
def clean_dates(df, date_column):
    df[date_column] = df[date_column].astype(str).apply(lambda x: re.sub(r'\.\d+', '', x))
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    return df

# Aplicar la función a cada DataFrame
sellin_anterior_melt = clean_dates(sellin_anterior_melt, 'Fecha')
fcbase_anterior_melt = clean_dates(fcbase_anterior_melt, 'Fecha')
fcadj_anterior_melt = clean_dates(fcadj_anterior_melt, 'Fecha')
cicloant_anterior_melt = clean_dates(cicloant_anterior_melt, 'Fecha')
stock_anterior_melt = clean_dates(stock_anterior_melt, 'Fecha')
noplan_anterior_melt = clean_dates(noplan_anterior_melt, 'Fecha')
fcfin_anterior_melt = clean_dates(fcfin_anterior_melt, 'Fecha')
fcval_anterior_melt = clean_dates(fcval_anterior_melt, 'Fecha')
fcacc_anterior_melt = clean_dates(fcacc_anterior_melt, 'Fecha')
sellout_anterior_melt = clean_dates(sellout_anterior_melt, 'Fecha')

# Imprimir los valores únicos de la columna Fecha después de la limpieza
print("Valores únicos de la columna 'Fecha' en sellin_anterior_melt:")
print(sellin_anterior_melt['Fecha'].unique())

print("Valores únicos de la columna 'Fecha' en fcbase_anterior_melt:")
print(fcbase_anterior_melt['Fecha'].unique())

print("Valores únicos de la columna 'Fecha' en fcadj_anterior_melt:")
print(fcadj_anterior_melt['Fecha'].unique())



In [ ]:
# Función para imprimir la cantidad de fechas únicas en la columna 'Fecha' de cada DataFrame
def imprimir_fechas_unicas(df, nombre_df):
    fechas_unicas = df['Fecha'].nunique()
    print(f"{nombre_df} tiene {fechas_unicas} fechas únicas")

# Lista de DataFrames y sus nombres
dataframes = [
    (sellin_anterior_melt, 'sellin_anterior_melt'),
    (fcbase_anterior_melt, 'fcbase_anterior_melt'),
    (fcadj_anterior_melt, 'fcadj_anterior_melt'),
    (cicloant_anterior_melt, 'cicloant_anterior_melt'),
    (stock_anterior_melt, 'stock_anterior_melt'),
    (noplan_anterior_melt, 'noplan_anterior_melt'),
    (fcfin_anterior_melt, 'fcfin_anterior_melt'),
    (fcval_anterior_melt, 'fcval_anterior_melt'),
    (fcacc_anterior_melt, 'fcacc_anterior_melt'),
    (sellout_anterior_melt, 'sellout_anterior_melt')
]

# Imprimir la cantidad de fechas únicas para cada DataFrame
for df, nombre in dataframes:
    imprimir_fechas_unicas(df, nombre)


In [ ]:
x=fcacc_anterior_melt['Fecha'].unique()
x